# 2. 
**Con el mismo $k$ implementa K-Medoids. Úsalo para el mismo dataset (`datasetblow.csv`)**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.spatial import Voronoi, voronoi_plot_2d

Se comienza por importar los datos del _dataset_

In [ ]:
dataset = pd.read_csv('datablow.csv', index_col=0)
dataset

In [ ]:
dataset_x, dataset_y = dataset["0"], dataset["1"]
dataset_fig, dataset_ax = plt.subplots()
dataset_fig.set_size_inches(11, 8.5)
dataset_ax.scatter(dataset_x, dataset_y)


In [ ]:
dataset_points = np.array(list(zip(dataset_x, dataset_y)))
dataset_points.shape

De igual manera se define $k$ como se definió en el primer ejercicio

In [ ]:
K=3

Con tal de resaltar las diferencias entre K-Medoids y K-Means, se implementó un algoritmo generalizado para _clustering_ `clustering_algorithm` que depende de:
* Una función de distancia
* Una función para obtener el centro de un cluster a partir de los puntos actuales

In [ ]:
def euclidean_distance(p1, p2):
    return np.linalg.norm(p1 - p2)


def assign_points_to_cluster(points, cluster_center_points):
    cluster_point_lists = [[] for _ in range(len(cluster_center_points))]
    points_cluster_indexes = np.zeros(len(points), dtype=int)
    for i, p in enumerate(points):
        closest_cluster_index = np.argmin(
            [euclidean_distance(p, c) for c in cluster_center_points], axis=0
        )
        points_cluster_indexes[i] = closest_cluster_index
        cluster_point_lists[closest_cluster_index].append(p)
    return list(map(np.array, cluster_point_lists)), points_cluster_indexes


def clustering_algorithm(
    num_clusters,
    points,
    cluster_center_function,
    seed=0,
):
    rng = np.random.default_rng(seed=seed)

    # Se inicializan los centros aleatoriamente
    cluster_center_points = points[
        rng.choice(points.shape[0], num_clusters, replace=False), :
    ]

    # Se actualizan en un ciclo hasta que convergen
    points_cluster_indexes = points_cluster_indexes_prev = None
    converged = False
    while not converged:
        # Se asigna cada punto a un cluster
        cluster_point_lists, points_cluster_indexes = assign_points_to_cluster(
            points, cluster_center_points
        )

        # Se calculan los nuevos centros
        for i in range(num_clusters):
            cluster_center_points[i] = cluster_center_function(cluster_point_lists[i])

        # Se revisa si hubo cambios en los centros
        if (points_cluster_indexes is not None) and np.array_equal(
            points_cluster_indexes, points_cluster_indexes_prev
        ):
            converged = True
        else:
            points_cluster_indexes_prev = points_cluster_indexes

    return cluster_center_points, cluster_point_lists


Con el método anterior se pueden definir los algoritmos K-Means y K-Medoids.

In [ ]:
def get_cluster_centroid(cluster_points):
    return np.mean(cluster_points, axis=0)


def k_means(num_clusters, points, seed=0):
    return clustering_algorithm(
        num_clusters,
        points,
        cluster_center_function=get_cluster_centroid,
        seed=seed,
    )


In [ ]:
def get_cluster_medoid(cluster_points):
    centroid = get_cluster_centroid(cluster_points)
    closest_point_index = np.argmin(
        [euclidean_distance(p, centroid) for p in cluster_points], axis=0
    )  # Se encuentra el índice del punto más cercano al centroide dentro del cluster
    return cluster_points[closest_point_index]


def k_medoids(num_clusters, points, seed=0):
    return clustering_algorithm(
        num_clusters,
        points,
        cluster_center_function=get_cluster_medoid,
        seed=seed,
    )


## (a)
**Comprueba que los centroides pertenecen al dataset**

Habiendo implementado K-Medoids, se aplica el algoritmo a nuestros datos y se verifica que los _medoids_ obtenidos forman parte del _dataset_.

In [ ]:
cluster_medoids, cluster_medoids_points_lists = k_medoids(K, dataset_points)
for m in cluster_medoids:
    print(f"({m[0]:.3f}, {m[1]:.3f}) in dataset points? {m in dataset_points}")

Mientras tanto, se puede notar que esto no es el caso con los centroides que se obtienen al utilizar K-Means.

In [ ]:
cluster_centroids, cluster_centroids_points_lists = k_means(K, dataset_points)
for m in cluster_centroids:
    print(f"({m[0]:.3f}, {m[1]:.3f}) in dataset points? {m in dataset_points}")

Se observa que los resultados de ambos algoritmos no son exactamente iguales.

In [ ]:
fig_compare, ax_compare = plt.subplots()
fig_compare.set_size_inches(11, 8.5)
ax_compare.scatter(dataset_x, dataset_y, label="Dataset points")
ax_compare.scatter(
    [m[0] for m in cluster_medoids],
    [m[1] for m in cluster_medoids],
    label="Cluster medoids",
)
ax_compare.scatter(
    [c[0] for c in cluster_centroids],
    [c[1] for c in cluster_centroids],
    label="Cluster centroids",
)
ax_compare.legend()


## (b)
**Genera el diagrama de Voronoi generado por los centroides**

Se hizo un método para generar gráficas con los resultados de los algoritmos de _clustering_ junto a los diagramas de Voronoi inducidos.

In [ ]:
color_series = [("#F2D7D5", "#C0392B"), ("#EBDEF0", "#9B59B6"), ("#D4E6F1", "#2980B9")]


def plot_clustering_results(cluster_centers, cluster_points_lists, padding):
    # Se genera diagrama de voronoi
    voronoi = Voronoi(cluster_centers)

    # Se crean vectores por coordenadas con los puntos de los clusters
    cluster_points_x = [[] for _ in range(len(cluster_centers))]
    cluster_points_y = [[] for _ in range(len(cluster_centers))]
    for i, cpl in enumerate(cluster_points_lists):
        cluster_points_x[i] += [p[0] for p in cpl]
        cluster_points_y[i] += [p[1] for p in cpl]

    # Se calculan los límites de la imagen
    min_x = min([min(c) for c in cluster_points_x])
    max_x = max([max(c) for c in cluster_points_x])
    min_y = min([min(c) for c in cluster_points_y])
    max_y = max([max(c) for c in cluster_points_y])

    # Se crean una figura y unos ejes para graficar
    fig, ax = plt.subplots()
    fig.set_size_inches(11, 8.5)

    # Se grafica cada cluster
    for i, c in enumerate(cluster_centers):
        ax.scatter(
            cluster_points_x[i],
            cluster_points_y[i],
            label=f"Cluster {i} points",
            c=color_series[i % len(color_series)][0],
        )

        ax.scatter(
            c[0],
            c[1],
            label=f"Cluster {i} origin",
            c=color_series[i % len(color_series)][1],
        )

    # Se grafican las fronteras del diagrama de voronoi
    voronoi_plot_2d(voronoi, ax, show_vertices=False, show_points=False)
    ax.set_xlim(min_x - padding, max_x + padding)
    ax.set_ylim(min_y - padding, max_y + padding)

    ax.legend()

    return fig, ax


Diagrama de Voronoi para los centroides generados por K-Medoids:

In [ ]:
plot_clustering_results(cluster_medoids, cluster_medoids_points_lists, padding=.1)

Diagrama de Voronoi para los centroides generados por K-Means:

In [ ]:
plot_clustering_results(cluster_centroids, cluster_centroids_points_lists, padding=.1)